##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Индивидуальное обучение с tf.distribute.Strategy

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/distribute/custom_training"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />Смотрите на TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ru/tutorials/distribute/custom_training.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Запустите в Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ru/tutorials/distribute/custom_training.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Изучайте код на GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ru/tutorials/distribute/custom_training.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Скачайте ноутбук</a>
  </td>
</table>

Note: Вся информация в этом разделе переведена с помощью русскоговорящего Tensorflow сообщества на общественных началах. Поскольку этот перевод не является официальным, мы не гарантируем что он на 100% аккуратен и соответствует [официальной документации на английском языке](https://www.tensorflow.org/?hl=en). Если у вас есть предложение как исправить этот перевод, мы будем очень рады увидеть pull request в [tensorflow/docs](https://github.com/tensorflow/docs) репозиторий GitHub. Если вы хотите помочь сделать документацию по Tensorflow лучше (сделать сам перевод или проверить перевод подготовленный кем-то другим), напишите нам на [docs-ru@tensorflow.org list](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs-ru).

В этом руководстве будет показано, как использовать [`tf.distribute.Strategy`](https://www.tensorflow.org/guide/distributed_training) с пользовательскими циклами обучения. Мы обучим простую модель CNN на наборе данных Fashion MNIST. Датасет Fashion MNIST содержит 60000 тренироваочных изображений размером 28x28 пикселей и 10000 тестовых изображений размером 28x28 пикселей.

Мы будем использовать кастомные циклы для обучения нашей модели, потому что они дают нам большую гибкость и больший контроль над обучением. Более того, при использовании кастомных циклов, - легче отлаживать модель и цикл обучения.

In [ ]:
# Import TensorFlow
import tensorflow as tf

# Helper libraries
import numpy as np
import os

print(tf.__version__)

## Загрузка датасета Fashion MNIST

In [ ]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# Добавление дополнительного измерения в массив -> новая размерность == (28, 28, 1)
# Мы делаем это, потому что первый слой в нашей модели - сверточный
# и требует ввода 4D (размер пакета, высота, ширина, каналы).
# размер пакета(batch_size) будет добавлен позже.
train_images = train_images[..., None]
test_images = test_images[..., None]

# Нормализуем пиксели изображения в диапазон [0, 1].
train_images = train_images / np.float32(255)
test_images = test_images / np.float32(255)

## Создание стратегии для распределенния переменных и графа

Как работает стратегия `tf.distribute.MirroredStrategy`?

* Все переменные и граф модели вычисляются на репликах.
* Входные данные равномерно распределяются по репликам.
* Каждая реплика вычисляет потери и градиенты для полученных входных данных.
* Градиенты синхронизируются по всем репликам путем их суммирования.
* После синхронизации такое же обновление производится для копий переменных на каждой реплике.


In [ ]:
# Если список устройств не указан в
# конструкторе `tf.distribute.MirroredStrategy`, он будет определенн автоматически.
strategy = tf.distribute.MirroredStrategy()

In [ ]:
print ('Колтчество доступных устройств: {}'.format(strategy.num_replicas_in_sync))

## Настройка входного конвейера

Экспортируйте граф и переменные в плтформо-независимый формат с помощью `SavedModel`. После сохранения модели вы можете загрузить ее как с контекстным менеджером, так и без него.

In [ ]:
BUFFER_SIZE = len(train_images)

BATCH_SIZE_PER_REPLICA = 64
GLOBAL_BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

EPOCHS = 10

Создайте датасеты и распределите их:

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).shuffle(BUFFER_SIZE).batch(GLOBAL_BATCH_SIZE) 
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(GLOBAL_BATCH_SIZE) 

train_dist_dataset = strategy.experimental_distribute_dataset(train_dataset)
test_dist_dataset = strategy.experimental_distribute_dataset(test_dataset)

## Создание модели

Создайте модель, используя `tf.keras.Sequential`. Также вы можете использовать наследование от `Model`.

In [ ]:
def create_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, 3, activation='relu'),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Conv2D(64, 3, activation='relu'),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dense(10)
    ])

  return model

In [ ]:
# Создайте директорию для сохранения чекпойнтов.
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")

## Определение функции потерь

Обычно на одной компьютере с 1-м GPU/CPU потери делятся на размер пакета входных данных(batch_size).

*Как же рассчитать потери при использовании tf.distribute.Strategy?*

* Например, предположим, что у вас 4 графических процессора и размер пакета 64. Один пакет ввода распределяется
  на 4 реплики (4 графических процессора), каждая реплика получает входящий пакет размером 16.

* Модель на каждой реплике(графический процессор) выполняет прямой проход с соответствующими входными данными и вычисляет потери. Теперь, вместо деления потерь на количество примеров в соответствующем вводе (BATCH_SIZE_PER_REPLICA = 16), потери следует разделить на GLOBAL_BATCH_SIZE (64).

*Зачем это делать?*

* Это необходимо сделать, потому что после расчета градиентов на каждой реплике они синхронизируются между репликами путем **суммирования**.

*Как это сделать в TensorFlow?*
* Если вы пишете свой собственный цикл обучения, как в этом руководстве, вам следует просуммировать потери для каждого примера и разделить сумму на GLOBAL_BATCH_SIZE:
  `scale_loss = tf.reduce_sum(потери) * (1. / GLOBAL_BATCH_SIZE)` или вы можете использовать `tf.nn.compute_average_loss`, который берет среднюю величину потерь на пакет, веса(если они используются) и GLOBAL_BATCH_SIZE в качестве аргументов и возвращает нормализованные потери.

* Если вы используете в своей модели потери регуляризации, вам необходимо масштабировать
  величина потерь по количеству реплик. Вы можете сделать это с помощью функции `tf.nn.scale_regularization_loss`.

* Использование `tf.reduce_mean` не рекомендуется. При расчете этим методом, потери делятся на фактический размер пакета реплик, который может меняться шаг за шагом.

* Уменьшение и масштабирование выполняется автоматически в файлах keras `model.compile` и` model.fit`

* При использовании классов `tf.keras.losses` (как в примере ниже) необходимо явно указать уменьшение потерь, которое может быть либо `NONE`, либо `SUM`. 
* `AUTO` и `SUM_OVER_BATCH_SIZE` запрещены при использовании с `tf.distribute.Strategy`. 
* `AUTO` запрещен, потому что пользователь должен четко указать, какое уменьшение  потерь он хочет использовать, и убедиться, что оно правильно работает в распределенной стратегии. `SUM_OVER_BATCH_SIZE` запрещен, потому что в текущей реализации он будет делиться только на размер пакета на реплике, а деление на количество реплик останется на усмотрение пользователя, что может быть легко упущено. Поэтому мы просим пользователя указать уменьшение потерь самостоятельно.
* Если метки являются многомерными, их необходимо усреднить по количеству элементов в каждой выборке - `per_example_loss`. Например, если форма `predictions` - `(batch_size, H, W, n_classes) `, а `labels` - `(batch_size, H, W)`, вам нужно будет обновить переменную `per_example_loss`: ` per_example_loss / = tf.cast (tf.reduce_prod(tf.shape(labels)[1:]), tf.float32)`


In [ ]:
with strategy.scope():
  # Установите tf.keras.losses.Reduction в None, чтобы мы могли сделать уменьшение позже 
  # и разделить его на глобальный размер пакета.
  loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=True,
      reduction=tf.keras.losses.Reduction.NONE)
  def compute_loss(labels, predictions):
    per_example_loss = loss_object(labels, predictions)
    return tf.nn.compute_average_loss(per_example_loss, global_batch_size=GLOBAL_BATCH_SIZE)

## Определение метрики для отслеживания потерь и точности

Эти метрики отслеживают потери валидации, а также точность обучения и валидации. 
Вы можете использовать метод `.result()` для получения накопленной статистики в любое время.

In [ ]:
with strategy.scope():
  test_loss = tf.keras.metrics.Mean(name='test_loss')

  train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
      name='train_accuracy')
  test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
      name='test_accuracy')

## Цикл обучения

In [ ]:
# модель, оптимайзер и чекпойнт должны быть созданы с использованием `strategy.scope`.
with strategy.scope():
  model = create_model()

  optimizer = tf.keras.optimizers.Adam()

  checkpoint = tf.train.Checkpoint(optimizer=optimizer, model=model)

In [ ]:
def train_step(inputs):
  images, labels = inputs

  with tf.GradientTape() as tape:
    predictions = model(images, training=True)
    loss = compute_loss(labels, predictions)

  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_accuracy.update_state(labels, predictions)
  return loss 

def test_step(inputs):
  images, labels = inputs

  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss.update_state(t_loss)
  test_accuracy.update_state(labels, predictions)

In [ ]:
# `strategy.run` реплицирует предоставленные расчеты и запускает их с распределенным вводом.
@tf.function
def distributed_train_step(dataset_inputs):
  per_replica_losses = strategy.run(train_step, args=(dataset_inputs,))
  return strategy.reduce(tf.distribute.ReduceOp.SUM, per_replica_losses,
                         axis=None)

@tf.function
def distributed_test_step(dataset_inputs):
  return strategy.run(test_step, args=(dataset_inputs,))

for epoch in range(EPOCHS):
  # ЦИКЛ ОБУЧЕНИЯ
  total_loss = 0.0
  num_batches = 0
  for x in train_dist_dataset:
    total_loss += distributed_train_step(x)
    num_batches += 1
  train_loss = total_loss / num_batches

  # ЦИКЛ ВАЛИДАЦИИ
  for x in test_dist_dataset:
    distributed_test_step(x)

  if epoch % 2 == 0:
    checkpoint.save(checkpoint_prefix)

  template = ("Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, "
              "Test Accuracy: {}")
  print (template.format(epoch+1, train_loss,
                         train_accuracy.result()*100, test_loss.result(),
                         test_accuracy.result()*100))

  test_loss.reset_states()
  train_accuracy.reset_states()
  test_accuracy.reset_states()

На что следует обратить внимание в приведенном выше примере:

* Мы перебираем наборы `train_dist_dataset` и `test_dist_dataset`, используя конструкцию `for x in ...`
* Масштабируемая потеря - это возвращаемое значение `distributed_train_step`. Это значение агрегируется по репликам с помощью вызова `tf.distribute.Strategy.reduce`, а затем по пакетам путем суммирования возвращаемого значения вызовов `tf.distribute.Strategy.reduce((tf.distribute.ReduceOp.SUM...`.
* `tf.keras.Metrics`и должны быть обновлены внутри train_step и test_step, которые выполняется в `tf.distribute.Strategy.run`.
* `tf.distribute.Strategy.run` возвращает результаты каждой локальной реплики в стратегии, и существует несколько способов получить результат. Вы можете выполнить `tf.distribute.Strategy.reduce`, чтобы получить агрегированное значение по всем репликам. Вы также можете выполнить `tf.distribute.Strategy.experimental_local_results`, чтобы получить список значений по каждой реплике и агрегировать его предпочитаемым способом.


## Восстановление последнего чекпойнта и тестирование

Модель, сохраненная с использованием `tf.distribute.Strategy` может быть восстановлена как с использованием `tf.distribute.Strategy`, так и без.

In [ ]:
eval_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
      name='eval_accuracy')

new_model = create_model()
new_optimizer = tf.keras.optimizers.Adam()

test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(GLOBAL_BATCH_SIZE)

In [ ]:
@tf.function
def eval_step(images, labels):
  predictions = new_model(images, training=False)
  eval_accuracy(labels, predictions)

In [ ]:
checkpoint = tf.train.Checkpoint(optimizer=new_optimizer, model=new_model)
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

for images, labels in test_dataset:
  eval_step(images, labels)

print ('Accuracy after restoring the saved model without strategy: {}'.format(
    eval_accuracy.result()*100))

## Альтернативные способы перебора набора данных

### Использование итераторов

Если вы хотите перебрать определенное количество шагов, а не весь набор данных, вы можете создать итератор, используя вызов `iter` и явный вызов `next` на итераторе. Вы можете перебирать датасет как внутри, так и вне tf.function. Вот небольшой фрагмент, демонстрирующий итерацию набора данных вне tf.function с использованием итератора.

In [ ]:
for _ in range(EPOCHS):
  total_loss = 0.0
  num_batches = 0
  train_iter = iter(train_dist_dataset)

  for _ in range(10):
    total_loss += distributed_train_step(next(train_iter))
    num_batches += 1
  average_train_loss = total_loss / num_batches

  template = ("Epoch {}, Loss: {}, Accuracy: {}")
  print (template.format(epoch+1, average_train_loss, train_accuracy.result()*100))
  train_accuracy.reset_states()

### Итерация внутри tf.function

Вы также можете перебирать весь `train_dist_dataset` внутри tf.function, используя конструкцию `for x in ... `или создавая итераторы, как мы делали выше. В приведенном ниже примере демонстрируется перенос одной эпохи обучения в tf.function и проход по `train_dist_dataset` внутри функции.

In [ ]:
@tf.function
def distributed_train_epoch(dataset):
  total_loss = 0.0
  num_batches = 0
  for x in dataset:
    per_replica_losses = strategy.run(train_step, args=(x,))
    total_loss += strategy.reduce(
      tf.distribute.ReduceOp.SUM, per_replica_losses, axis=None)
    num_batches += 1
  return total_loss / tf.cast(num_batches, dtype=tf.float32)

for epoch in range(EPOCHS):
  train_loss = distributed_train_epoch(train_dist_dataset)

  template = ("Epoch {}, Loss: {}, Accuracy: {}")
  print (template.format(epoch+1, train_loss, train_accuracy.result()*100))

  train_accuracy.reset_states()

### Отслеживание потерь при обучении по репликам

Примечание. Как правило, вы должны использовать `tf.keras.Metrics` для отслеживания значений по выборке и избегать значений, которые были агрегированы в реплике.

Мы **не** рекомендуем использовать `tf.keras.Metrics` для отслеживания потерь при обучении по разным репликам из-за выполнения масштабирования расчитанных потерь.

Например, если вы выполняете учебное задание со следующими характеристиками:
* Две реплики
* На каждой реплике обрабатываются два образца
* Полученные значения потерь: [2, 3] и [4, 5] на каждой реплике.
* Глобальный размер пакета = 4

При масштабировании потерь вы вычисляете значение потерь для пакета на каждой реплике, добавляя значения потерь и затем делите его на глобальный размер пакета. В данном случае: `(2 + 3) / 4 = 1,25` и` (4 + 5) / 4 = 2,25`.

Если вы используете `tf.keras.Metrics` для отслеживания потерь в двух репликах, результат будет другим. В этом примере вы получаете `total`, равный 3,50, и `count`, равный 2, в результате чего `total`/`count` = 1,75 при вызове `result()` для метрики. Потери, рассчитанные с помощью `tf.keras.Metrics`, масштабируются с помощью дополнительного коэффициента, который равен количеству синхронизированных реплик.

### Руководства и примеры

Вот несколько примеров использования стратегии распределния с пользовательскими циклами обучения:

1. [Руководство по распределенному обучению](../../guide/distributed_training)
2. [DenseNet](https://github.com/tensorflow/examples/blob/master/tensorflow_examples/models/densenet/distributed_train.py) в примере используется `MirroredStrategy`.
1. [BERT](https://github.com/tensorflow/models/blob/master/official/nlp/bert/run_classifier.py) пример обучался с использованием `MirroredStrategy` и `TPUStrategy`.
Этот пример особенно полезен для понимания того, как выполнять загрузку из контрольной точки и создавать периодические контрольные точки во время распределенного обучения.
2. [NCF](https://github.com/tensorflow/models/blob/master/official/recommendation/ncf_keras_main.py) пример обучался с использованием `MirroredStrategy`, которая может быть включена установкой флага `keras_use_ctl`.
3. [NMT](https://github.com/tensorflow/examples/blob/master/tensorflow_examples/models/nmt_with_attention/distributed_train.py) пример обучался с использованием `MirroredStrategy`.

Дополнительные примеры перечислены в [Руководстве по распределенной стратегии](../../guide/distributed_training.ipynb#examples_and_tutorials).

## Что дальше

* Попробуйте новый API `tf.distribute.Strategy` на своих моделях.
* Посетите [Раздел производительности](../../guide/function.ipynb) в руководстве, чтобы узнать больше о других стратегиях и [инструментах](../../guide/profiler.md), которые можно использовать для улучшения производительности ваших моделей TensorFlow.